In [2]:
import pandas as pd
import polars as pl
from textblob import TextBlob
import datashader as ds
import colorcet as cc
import matplotlib.pyplot as plt
import yfinance as yf
import re
plt.style.use('fivethirtyeight')
from transformers import BertTokenizer, BertModel
from transformers import get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
import numpy as np
import emoji
import os
import snscrape.modules.twitter as sntwitter
plt.style.use('fivethirtyeight')

In [ ]:
query = 'S&P500 lang:en until:2022-12-31 since:2022-07-01'
tweets=[]
limit=1000000

In [ ]:
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.id, tweet.rawContent, tweet.likeCount, tweet.retweetCount, tweet.user.followersCount])
    
df = pd.DataFrame(tweets, columns=['Date', 'Id', 'Tweet', 'Like', 'Retweet', 'Followers'])

In [2]:
data = pd.read_pickle('tweets_07_01_12_31_newall.pkl')

In [3]:
def Preprocess_Tweets(data):
		
	data['Text_Cleaned'] = data['Tweet'].str.lower()

	## FIX HYPERLINKS
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'https?:\/\/.*[\r\n]*', ' ',regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'www.*[\r\n]*', ' ',regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('https', '', regex=False)


	## FIX INDIVIDUAL SYMBOLS 
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(': ', ' ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(', ', ' ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('. ', ' ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[;\n~]', ' ', regex=True)

	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace("[]'â€¦*™|]", '', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[[()!?"]', '', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('_', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('w/', ' with ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('f/', ' for ', regex=False)
    
	## FIX EMOJIS
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(':)', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(':-)', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(':(', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(':-(', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('0_o', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(';)', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('=^.^=', '', regex=False)


	## FIX % SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('%', ' percent ', regex=False)
    
	## FIX 【】 SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('】', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('【', '', regex=False)    


	## FIX & SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' & ', ' and ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('&amp', ' and ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('&gt', ' greater than ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('cup&handle', 'cup and handle', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('c&h', 'cup and handle', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('head&shoulders', 'head and shoulders', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('h&s', 'head and shoulders', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('point&figure', 'point and figure', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('p&f', 'point and figure', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('s&p', 'SP500', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('q&a', 'question and answer', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('&', ' and ', regex=False)


	## FIX USER TAGS AND HASTAGS
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('@[a-z0-9]+', '', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('#[a-z0-9]+', '', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('@', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('#', '', regex=False)
	   
		
	## FIX EMBEDDED COMMAS AND PERIODS    
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z]),([a-z])', r'\1 \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]),([0-9])', r'\1\2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])[+]+', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(',', '', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('u.s.', ' us ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('\.{2,}', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z])\.([a-z])', r'\1 \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('pdating', 'updating', regex=False) 
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z])\.', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'\.([a-z])', r' \1', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' . ', ' ', regex=False)
		

	## FIX + SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'[+]([0-9])', r'positive \1', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('c+h', 'cup and handle', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('h+s', 'head and shoulders', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('cup+handle', 'cup and handle', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' + ', ' and ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('+ ', ' ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z])[+]([a-z])', r'\1 and \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('+', '', regex=False)



		
	## FIX - SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z])[-]+([a-z])', r'\1 \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z]) - ([a-z])', r'\1 to \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]) -([0-9\.])', r'\1 to \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r' [-]([0-9])', r' negative \1', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])-([0-9\.])', r'\1 to \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]) - ([0-9\.])', r'\1 to \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9a-z])-([0-9a-z])', r'\1 \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[-]+[>]', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' [-]+ ', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('-', ' ', regex=False)



	## FIX $ SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[$][0-9\.]', ' dollars ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('$', '', regex=False)


	## FIX = SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('=', ' equals ', regex=False)

		
	## FIX / SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('b/c', ' because ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('b/out', ' break out ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('b/o', ' break out ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('p/e', ' pe ratio ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' [/]+ ', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 1/2 ', ' .5 ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 1/4 ', ' .25 ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 3/4 ', ' .75 ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 1/3 ', ' .3 ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 2/3 ', ' .6 ', regex=False)

	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[/]{2,}', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([a-z])/([a-z])', r'\1 and \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[0-9]+/[0-9]+/[0-9]+', '', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]{3,})/([0-9\.]{2,})', r'\1 to \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]{2,})/([0-9\.]{3,})', r'\1 to \2', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[a-z0-9]+/[a-z0-9]+', ' ', regex=True)

	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('/', '', regex=False)


	## FIX < > SYMBOLS
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[<]+ ', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('<', ' less than ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' [>]+', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('>', ' greater than ', regex=False)


	## FIX : SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[0-9]+:[0-9]+am', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('[0-9]+:[0-9]', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(':', ' ', regex=False)
    
    ## FIX " SYMBOL
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('“', ' ', regex=False)
	
    ## FIX UNITS
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('user ', ' ', regex=False)

	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]+)dma', r'\1 displaced moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'dma([0-9]+)', r'\1 displaced moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]+)sma', r'\1 simple moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'sma([0-9]+)', r'\1 simple moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]+)ema', r'\1 expontential moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'ema([0-9]+)', r'\1 expontential moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9]+)ma', r'\1 moving average ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'ma([0-9]+)', r'\1 moving average ', regex=True)

	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])mos', r'\1 months ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])minute', r'\1 minute ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])minutes', r'\1 minutes ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])min', r'\1 minute ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])mins', r'\1 minutes ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])day', r'\1 day ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])days', r'\1 days ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])wk', r'\1 week ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' wk ', ' week ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' wknd ', ' weekend ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])wks', r'\1 weeks ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])hours', r'\1 hours ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])hour', r'\1 hour ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])yr', r'\1 year ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])yrs', r'\1 years ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' yr', ' year ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])am', r'\1 am ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])pm', r'\1 pm ', regex=True)

	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])est', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])ish', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9 ])pts', r'\1 points ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])x', r'\1 times ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])th', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])rd', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])st', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])nd', r'\1 ', regex=True)

	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('mrkt', 'market', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' vol ', ' volume ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' ptrend', ' positive trend ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' ppl', ' people ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' pts', ' points ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' pt', ' point ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' l(ol){1,}', ' laugh ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('imho', ' in my opinion ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace('prev ', 'previous ', regex=True)


	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 1q', ' first quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 2q', ' second quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 3q', ' third quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 4q', ' fourth quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' q1', ' first quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' q2', ' second quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' q3', ' third quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' q4', ' fourth quarter ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' 10q ', ' form 10 ', regex=False)

	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])million', r'\1 million ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])mil', r'\1 million ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' mil ', ' million ', regex=False)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])billion', r'\1 billion ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])cents', r'\1 cents ', regex=True)

	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])3d', r'\1 3 dimensional ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])gb', r'\1 3 gigabytes ', regex=True)



	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])c', r'\1 calls ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])y', r'\1 year ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])p', r'\1 puts ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])d', r'\1 days ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])h', r'\1 hour ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])s', r'\1 ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])k1', r'\1 thousand ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])k', r'\1 thousand ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])m', r'\1 million ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])b', r'\1 billion ', regex=True)

		
	data['Text_Cleaned'] = data['Text_Cleaned'].replace(r'([0-9])([a-z])', r'\1 \2', regex=True)

	## FIX EXTRA SPACES AND ENDING PUNCTUATION
	data['Text_Cleaned'] = data['Text_Cleaned'].str.replace(' +', ' ', regex=True)
	data['Text_Cleaned'] = data['Text_Cleaned'].str.strip(' .!?,)(:-')


	return data

In [4]:
# Process the Tweets for NLP
data = Preprocess_Tweets(data)
display(data)

D:\anaconda3\lib\site-packages\pandas\core\strings\object_array.py:158: FutureWarning: Possible nested set at position 1
  pat = re.compile(pat, flags=flags)


,Date,Id,Tweet,Like,Retweet,Followers,Text_Cleaned
0,2022-12-30 23:48:49+00:00,1608973410788319234,NC State gets worked at Clemson #breaking #bre...,0,0,267,nc state gets worked at clemson
1,2022-12-30 23:48:48+00:00,1608973408859049987,Ancelotti: “An unforgettable year is coming to...,0,0,267,ancelotti an unforgettable year is coming to a...
2,2022-12-30 23:48:47+00:00,1608973405616750592,Nunez helps Liverpool push Leicester past thei...,0,1,267,nunez helps liverpool push leicester past thei...
3,2022-12-30 23:48:45+00:00,1608973397597499395,"Btw $GMX P/E is 13, think about it 🤔\np.s. SP5...",0,0,280,btw gmx pe ratio is 13 think about it 🤔 p s sp...
4,2022-12-30 23:46:31+00:00,1608972835409760256,Your Evening Briefing: S&amp;P 500 Ends a Grim...,1,0,3596,your evening briefing s and p 500 ends a grim ...
...,...,...,...,...,...,...,...
218723,2022-07-01 00:19:35+00:00,1542664174198718465,The S&amp;P500 total return of Bloomberg Globa...,0,0,31,the s and p500 total return of bloomberg globa...
218724,2022-07-01 00:17:24+00:00,1542663628121317376,"Pitch: For 2 and 20, we will match the sp500 r...",0,0,1230,pitch for 2 and 20 we will match the sp500 ret...
218725,2022-07-01 00:16:38+00:00,1542663432020938752,"I prolly shouldn’t say this, but I underperfor...",0,0,3218,i prolly shouldn’t say this but i underperform...
218726,2022-07-01 00:05:01+00:00,1542660508649525248,Top stocks with TA score trending DOWN (SP500)...,0,0,5479,top stocks with ta score trending down sp500


In [3]:
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
# Prepare Vader sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Predict sentiment with Vader classifier
data['Vader_Scores'] = data['Text_Cleaned'].apply(lambda score: sid.polarity_scores(score)['compound'])
data['Vader_Prediction'] = data['Vader_Scores'].apply(lambda score: 1 if score >=0 else -1)

In [5]:
data = data.drop('Vader_Scores', axis=1)
data = data.rename(columns={'Vader_Prediction':'Sentiment'})

In [2]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

# Create list of custom stop words to remove
StopWords = set([s.replace("'", '') for s in stopwords.words('english') if s not in ['not', 'up', 'down', 'above', 'below', 'under', 'over']])


In [6]:
# Remove stop words for tweets
data['Text_Processed'] = data['Text_Cleaned'].apply(lambda s: " ".join([word for word in s.split() if word not in StopWords]))
data['Text_Processed'] = data['Text_Processed'].str.strip()

In [6]:
 data = data.loc[(data['Date']>='2022-10-31') & (data['Date']<='2022-12-31')]

In [7]:
data

,Date,Id,Tweet,Like,Retweet,Followers,Text_Cleaned,Text_Processed,Sentiment
0,2022-12-30 23:48:49+00:00,1608973410788319234,NC State gets worked at Clemson #breaking #bre...,0,0,267,nc state gets worked at clemson,nc state gets worked clemson,1
1,2022-12-30 23:48:48+00:00,1608973408859049987,Ancelotti: “An unforgettable year is coming to...,0,0,267,ancelotti an unforgettable year is coming to a...,ancelotti unforgettable year coming end style”,1
2,2022-12-30 23:48:47+00:00,1608973405616750592,Nunez helps Liverpool push Leicester past thei...,0,1,267,nunez helps liverpool push leicester past thei...,nunez helps liverpool push leicester past brea...,1
3,2022-12-30 23:48:45+00:00,1608973397597499395,"Btw $GMX P/E is 13, think about it 🤔\np.s. SP5...",0,0,280,btw gmx pe ratio is 13 think about it 🤔 p s sp...,btw gmx pe ratio 13 think 🤔 p sp500 average pe...,1
4,2022-12-30 23:46:31+00:00,1608972835409760256,Your Evening Briefing: S&amp;P 500 Ends a Grim...,1,0,3596,your evening briefing s and p 500 ends a grim ...,evening briefing p 500 ends grim year down 20 ...,-1
...,...,...,...,...,...,...,...,...,...
145976,2022-10-31 00:13:10+00:00,1586873882425761793,"Instant Analysis: Giants fall to Seahawks, ent...",0,0,268,instant analysis giants fall to seahawks enter...,instant analysis giants fall seahawks enter by...,1
145977,2022-10-31 00:13:10+00:00,1586873879502503937,How to Watch The Simpsons' Death Note Episode ...,0,0,268,how to watch the simpsons death note episode,watch simpsons death note episode,-1
145978,2022-10-31 00:13:09+00:00,1586873877132562432,"Henry runs for 219 yards, 2 TDs as Titans down...",0,0,268,henry runs for 219 yards 2 tds as titans down ...,henry runs 219 yards 2 tds titans down texans ...,1
145979,2022-10-31 00:06:24+00:00,1586872176820502529,#dailyupdate #sp500 #wallstreet #markettiming ...,1,0,346,e mini s and p 500 es dec it was d...,e mini p 500 es dec daring make prediction es ...,1


In [8]:
# Split the training and test data into 80/20 split
train_pct = .8
np.random.seed(1)
idx = np.random.permutation(len(data))

X_train = data['Text_Processed'].values[idx[:int(train_pct*len(data))]]
y_train = data['Sentiment'].values[idx[:int(train_pct*len(data))]]
y_train[y_train==-1] = 0
X_test = data['Text_Processed'].values[idx[int(train_pct*len(data)):]]
y_test = data['Sentiment'].values[idx[int(train_pct*len(data)):]]
y_test[y_test==-1] = 0

## Bert Sentiment Analysis

In [4]:
# Prepare the Bert NLP model tokenizer to encode tweets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [6]:
# determine max length for encoding
# encoded = [tokenizer.encode(sent, add_special_tokens=True) for sent in data['Text_Processed'].values]
# MAX_LEN = max([len(sent) for sent in encoded])
# print('Max length: ', MAX_LEN)
MAX_LEN=148

In [7]:
# Encode the tweets for Bert model
def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []
    
    # For each tweet
    for line in data:
        # encode the data. Return input encoding and attention mask
        encoding = tokenizer.encode_plus(
                text=line, # data to process
                add_special_tokens=True, # adds special chars [CLS] and [SEP] to encoding 
                padding='max_length', # pad the tweets with 0s to fit max length
                max_length = MAX_LEN, # assign max length
                truncation=True, # truncate tweets longer than max length
                return_tensors="pt", # return tensor as pytorch tensor
                return_attention_mask=True # return the attention mask
                )

        # add the encodings to the list
        input_ids.append(encoding.get('input_ids'))
        attention_masks.append(encoding.get('attention_mask'))
    
    # return the lists as tensors
    input_ids = torch.concat(input_ids)
    attention_masks = torch.concat(attention_masks)
    
    return input_ids, attention_masks

In [11]:
# Define the Bert NLP Classifier
class BertClassifier(nn.Module):
    def __init__(self, freeze=False):
        super(BertClassifier, self).__init__()

        # Define the neurons for the final layer
        input_layer = 768
        hidden_layer = 50
        output_layer = 2

        # Use the pretrained Bert model for first section of NN
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Define a final layer to attach to the Bert model for custom classification
        self.classifier = nn.Sequential(
            nn.Linear(input_layer, hidden_layer), 
            nn.ReLU(), 
            nn.Linear(hidden_layer, output_layer))

        # Freeze the model from updating
        if freeze:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    # Return classification from Bert model 
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask)
        h_cls = outputs[0][:, 0, :]
        logits = self.classifier(h_cls)

        return logits

In [13]:
# Encode the train and test data for Bert
X_train_inputs, X_train_masks = preprocessing_for_bert(X_train)
X_test_inputs, X_test_masks = preprocessing_for_bert(X_test)

# Get the train and test labels
y_train_labels = torch.tensor(y_train)
y_test_labels = torch.tensor(y_test)

print(X_train_inputs.shape, X_train_masks.shape, y_train_labels.shape)
print(X_test_inputs.shape, X_test_masks.shape, y_test_labels.shape)

# Set batch size to 16. recommended 16 or 32 depending on GPU size 
batch_size = 16

# Randomize the train data and define dataloader for model training 
train_data = TensorDataset(X_train_inputs, X_train_masks, y_train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Randomize the test data and define dataloader for model testing
test_data = TensorDataset(X_test_inputs, X_test_masks, y_test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

torch.Size([63092, 148]) torch.Size([63092, 148]) torch.Size([63092])
torch.Size([15773, 148]) torch.Size([15773, 148]) torch.Size([15773])


In [14]:
# Set random seed for repeatability
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)

# Check if GPU is available and assign device 
if torch.cuda.is_available():       
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
# Initialize Bert Classifier
model = BertClassifier(freeze=False)

# Send model to device (GPU if available)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [15]:
# Define model hyperparameters
epochs = 4
steps = len(train_dataloader) * epochs
learning_rate = 5e-5
epsilon = 1e-8

# Define Adam optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=epsilon)

# Define scheduler for training the optimizer 
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

# Define cross entropy loss function 
loss_function = nn.CrossEntropyLoss()

In [16]:
# For the number of epochs
for e in range(epochs):
    # Assign model to train
    model.train()

    # Intialize loss to zero
    train_loss = 0
    
    # For each batch
    for batch in train_dataloader:
        # Get batch inputs, masks and labels 
        batch_inputs, batch_masks, batch_labels = batch
        
        # Send variables to device (GPU if available)
        batch_inputs = batch_inputs.to(device)
        batch_masks = batch_masks.to(device)
        batch_labels = batch_labels.to(device)

        # Reset the model gradient
        model.zero_grad()

        # Get classification of encoded values
        logits = model(batch_inputs, batch_masks)
        
        # Calculate loss based on predictions and known values
        loss = loss_function(logits, batch_labels)
        
        # Add loss to the running total
        train_loss += loss.item()
        
        # Update the model weights based on the loss 
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over batch
    train_loss /= len(train_dataloader)
    
    # Assign the model to evaluate    
    model.eval()

    # Initialize losses
    test_loss = 0
    test_acc = 0

    # For each batch
    for batch in test_dataloader:
        # Get encoding inputs, masks and labels
        batch_inputs, batch_masks, batch_labels = batch
        
        # Send variables to device (GPU if available)
        batch_inputs = batch_inputs.to(device)
        batch_masks = batch_masks.to(device)
        batch_labels = batch_labels.to(device)

        # Predict the input values without updating the model 
        with torch.no_grad():
            logits = model(batch_inputs, batch_masks)

        # Calculate the loss
        loss = loss_function(logits, batch_labels)
        test_loss += loss.item()

        # Convert predictions to 0 and 1
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate accuracy of model on test data 
        accuracy = (preds == batch_labels).cpu().numpy().mean() * 100
        test_acc += accuracy

    # Calculate average loss and accuracy per each batch
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

    # Print epoch information 
    print('Epoch: %d  |  Train Loss: %1.5f  |  Test Loss: %1.5f  |  Test Accuracy: %1.2f'%(e+1, train_loss, test_loss, test_acc))
    
# Save model
torch.save(model.state_dict(), 'stock_sentiment_model.pt')

Epoch: 1  |  Train Loss: 0.20956  |  Test Loss: 0.14960  |  Test Accuracy: 95.74
Epoch: 2  |  Train Loss: 0.08687  |  Test Loss: 0.10034  |  Test Accuracy: 97.64
Epoch: 3  |  Train Loss: 0.03559  |  Test Loss: 0.09699  |  Test Accuracy: 98.42
Epoch: 4  |  Train Loss: 0.01392  |  Test Loss: 0.09399  |  Test Accuracy: 98.59


## Bert Sentiment Classification

In [8]:
stock = pd.read_pickle('tweets_01_01_03_31.pickle')

In [9]:
stock = Preprocess_Tweets(stock)
display(stock)

D:\anaconda3\lib\site-packages\pandas\core\strings\object_array.py:158: FutureWarning: Possible nested set at position 1
  pat = re.compile(pat, flags=flags)


,Date,Id,Tweet,Like,Retweet,Followers,Text_Cleaned
0,2023-03-30 23:59:43+00:00,1641591062845063172,Excited for a new month! April 2023 is just ar...,0,0,490,excited for a new month april 2023 is just aro...
1,2023-03-30 23:56:56+00:00,1641590365185605633,LIVE streaming FREE #GOLD #XAUUSD SIGNALS\nPle...,0,1,211,live streaming free signals please like and su...
2,2023-03-30 23:54:04+00:00,1641589643324915713,$DXY:\n\nBearish Head &amp; Shoulders pattern ...,3,0,337,dxy bearish head and shoulders pattern forming...
3,2023-03-30 23:53:03+00:00,1641589387682263040,"【🇺🇸S&amp;P500】 +0.57% (+23.02) 4,050.83 https...",0,0,1025,🇺🇸s and p500 positive 0.57 percent positive 23...
4,2023-03-30 23:49:47+00:00,1641588562448752640,@Ben_Egli @GovTinaKotek Cool. What will the SP...,0,0,46,cool what will the sp500 close at tomorrow
...,...,...,...,...,...,...,...
57067,2023-01-01 00:08:06+00:00,1609340652885524482,'The other team is a professional football tea...,0,0,269,the other team is a professional football team...
57068,2023-01-01 00:08:05+00:00,1609340650687709184,Jessie James Decker Shares Stunning Snaps From...,0,0,269,jessie james decker shares stunning snaps from...
57069,2023-01-01 00:07:20+00:00,1609340459129749505,@SamanthaLaDuc All above 3400. Nice.\n\nWe hav...,0,0,35,all above 3400 nice we have a recession coming...
57070,2023-01-01 00:06:19+00:00,1609340203076046850,@CurtisEyiogbe @mbontrager5 -8% for SCV vs -15...,0,0,2891,negative 8 percent for scv vs negative 15 perc...


In [1]:
# Prepare the Bert NLP model tokenizer to encode tweets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
MAX_LEN=148

model = BertClassifier()
model.load_state_dict(torch.load('stock_sentiment_model.pt'))

# Check if GPU is available and assign device 
if torch.cuda.is_available():       
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
# Send model to device (GPU if available)
model.to(device)

stock_inputs, stock_masks = preprocessing_for_bert(stock['Text_Cleaned'].values)

batch_size = 16
 
stock_data = TensorDataset(stock_inputs, stock_masks)
stock_sampler = RandomSampler(stock_data)
stock_dataloader = DataLoader(stock_data, sampler=stock_sampler, batch_size=batch_size)

model.eval()
predictions = []

# For each batch
for batch in stock_dataloader:
    # Get encoding inputs, masks and labels
    batch_inputs, batch_masks = batch

    # Send variables to device (GPU if available)
    batch_inputs = batch_inputs.to(device)
    batch_masks = batch_masks.to(device)

    # Predict the input values without updating the model 
    with torch.no_grad():
        logits = model(batch_inputs, batch_masks)

    # Convert predictions to 0 and 1
    preds = torch.argmax(logits, dim=1).flatten()
    predictions.append(preds)

# Combine all batch predictions
predictions = torch.cat(predictions).cpu().numpy()

# Add predictions to stock dataframe
stock['Sentiment'] = predictions

stock.to_pickle('stock_data_sentiment.pickle')

NameError: name 'BertTokenizer' is not defined

## Predict the stock market

In [15]:
stock['Date'] = pd.to_datetime(stock['Date'])

stock.loc[stock['Sentiment'] == 0, 'Sentiment'] = -1



,Date,Id,Tweet,Like,Retweet,Followers,Text_Cleaned,Sentiment
0,2023-03-30 23:59:43+00:00,1641591062845063172,Excited for a new month! April 2023 is just ar...,0,0,490,excited for a new month april 2023 is just aro...,1
1,2023-03-30 23:56:56+00:00,1641590365185605633,LIVE streaming FREE #GOLD #XAUUSD SIGNALS\nPle...,0,1,211,live streaming free signals please like and su...,0
2,2023-03-30 23:54:04+00:00,1641589643324915713,$DXY:\n\nBearish Head &amp; Shoulders pattern ...,3,0,337,dxy bearish head and shoulders pattern forming...,1
3,2023-03-30 23:53:03+00:00,1641589387682263040,"【🇺🇸S&amp;P500】 +0.57% (+23.02) 4,050.83 https...",0,0,1025,🇺🇸s and p500 positive 0.57 percent positive 23...,1
4,2023-03-30 23:49:47+00:00,1641588562448752640,@Ben_Egli @GovTinaKotek Cool. What will the SP...,0,0,46,cool what will the sp500 close at tomorrow,1
...,...,...,...,...,...,...,...,...
57067,2023-01-01 00:08:06+00:00,1609340652885524482,'The other team is a professional football tea...,0,0,269,the other team is a professional football team...,1
57068,2023-01-01 00:08:05+00:00,1609340650687709184,Jessie James Decker Shares Stunning Snaps From...,0,0,269,jessie james decker shares stunning snaps from...,1
57069,2023-01-01 00:07:20+00:00,1609340459129749505,@SamanthaLaDuc All above 3400. Nice.\n\nWe hav...,0,0,35,all above 3400 nice we have a recession coming...,1
57070,2023-01-01 00:06:19+00:00,1609340203076046850,@CurtisEyiogbe @mbontrager5 -8% for SCV vs -15...,0,0,2891,negative 8 percent for scv vs negative 15 perc...,1
